In [1]:
import os
import torch

from fairseq.models.bart import BARTModel
from rouge import Rouge, FilesRouge
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

ImportError: cannot import name 'FilesRouge'

In [ ]:
import numpy as np

from rouge import Rouge, FilesRouge
from tqdm import tqdm as tqdm
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score

def create_hypotheses(weights_path, data_path, test_hypo):
    bart = BARTModel.from_pretrained(
        weights_path,
        checkpoint_file='checkpoint_best.pt',
        data_name_or_path=data_path
    )

    bart.eval()
    bart.cuda()
    count = 1
    bsz = 8
    test_trans = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_trans_cons_label_2.source'
    test_c99 = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_c99_label.source'
    with open(test_trans) as source, open(test_c99) as source2, open(test_hypo, 'wt', encoding='utf-8') as fout:
        s1 = source.readlines()
        s2 = source2.readlines()

        slines = [s1[0].strip()]
        slines2 = [s2[0].strip()]

        for i in tqdm(range(1, len(s1))):
            if count % bsz == 0:
                with torch.no_grad():
                    hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
                    torch.cuda.empty_cache()

                for hypothesis in hypotheses_batch:
                    fout.write(hypothesis + '\n')
                    fout.flush()
                slines = []
                slines2 = []

            slines.append(s1[i].strip())
            slines2.append(s2[i].strip())

            count += 1

        if slines != []:

            hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)


            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + '\n')
                fout.flush()

    
def load_ref_and_hypo(hyp_path, tokenize=False):
    ref_path = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_trans_cons_label_2.target'
    hypotheses = []
    with open(hyp_path, 'r') as f:
        lines = f.readlines()
        for l in lines:
            hypotheses.append(l[:-1])

    reference = []
    with open(ref_path, 'r') as f:
        lines = f.readlines()
        for l in lines:
            reference.append(l[:-1])

    if tokenize:
        hypotheses = [hypo.split() for hypo in hypotheses]
        reference = [ref.split() for ref in reference]

    return hypotheses, reference
    
    
def get_rouge(hyp_path):
    hypothesis, reference = load_ref_and_hypo(hyp_path)
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference, avg = True)
    #print('Test', rouge.get_scores(hypothesis, reference, avg = True))

    print('TEST')
    for key, metric in scores.items():
        print(key)
        for k, v in metric.items():
            print(f'{k}: {v}')
        

def get_bleu(hyp_path):
    hypotheses, reference = load_ref_and_hypo(hyp_path, tokenize=True)
    print('Corpus BLEU 1-gram: %f' % corpus_bleu(reference, hypotheses, weights=(1, 0, 0, 0)))
    print('Corpus BLEU 2-gram: %f' % corpus_bleu(reference, hypotheses, weights=(0.5, 0.5, 0, 0)))
    print('Corpus BLEU 3-gram: %f' % corpus_bleu(reference, hypotheses, weights=(0.33, 0.33, 0.33, 0)))
    print('Corpus BLEU 4-gram: %f' % corpus_bleu(reference, hypotheses, weights=(0.25, 0.25, 0.25, 0.25)))


def get_meteor(hyp_path):
    hypotheses, reference = load_ref_and_hypo(hyp_path, tokenize=True)
    reference = [[ref] for ref in reference]
    meteor_score = corpus_meteor(reference, hypotheses)
    print('METEOR: %f' % meteor_score)

    
def corpus_meteor(expected, predicted):
    meteor_score_sentences_list = list()
    [meteor_score_sentences_list.append(meteor_score(expect, predict)) for expect, predict in zip(expected, predicted)]
    meteor_score_res = np.mean(meteor_score_sentences_list)
    return meteor_score_res

In [3]:
weights_path = 'checkpoints_multi_view'
data_path = './cnn_dm-bin_2'
test_hypo_pretrained = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best_PRETRAINED.hypo'

In [4]:
create_hypotheses(weights_path, data_path, test_hypo_pretrained)

here!


100%|██████████| 499/499 [05:57<00:00,  1.40it/s]


In [40]:
weights_path = 'train_sh/checkpoints_stage'
data_path = '../dialogsum-bin'
test_hypo_single = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best_SINGLE.hypo'

In [41]:
create_hypotheses(weights_path, data_path, test_hypo_single)

here!


100%|██████████| 499/499 [06:41<00:00,  1.24it/s]


In [42]:
weights_path = 'train_sh/checkpoints'
data_path = '../dialogsum-bin_2'
test_hypo_multi = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best_MULTI.hypo'

In [44]:
create_hypotheses(weights_path, data_path, test_hypo_multi)

here!


100%|██████████| 499/499 [07:05<00:00,  1.17it/s]


In [50]:
for path in [test_hypo_pretrained, test_hypo_single, test_hypo_multi]:
    print(path.split('/')[-1])
    get_rouge(path)
    get_bleu(path)
    get_meteor(path)
    print('\n')

test_best_multi_attn_best_PRETRAINED.hypo
TEST
rouge-1
r: 0.42381762924534044
p: 0.3106988218727275
f: 0.34378587073025224
rouge-2
r: 0.16600478963012985
p: 0.12211266858958905
f: 0.13351775470396782
rouge-l
r: 0.3963902365567834
p: 0.2902285889817407
f: 0.3213212855506028
Corpus BLEU 1-gram: 0.025165
Corpus BLEU 2-gram: 0.001339
Corpus BLEU 3-gram: 0.000000
Corpus BLEU 4-gram: 0.000000
METEOR: 0.284844


test_best_multi_attn_best_SINGLE.hypo
TEST
rouge-1
r: 0.5678239788779302
p: 0.3457162632182704
f: 0.4214851315520334
rouge-2
r: 0.24301067687032568
p: 0.13907783503136428
f: 0.1724773875682337
rouge-l
r: 0.5112094712565164
p: 0.31107723812728544
f: 0.3794061540863322
Corpus BLEU 1-gram: 0.024588
Corpus BLEU 2-gram: 0.000000
Corpus BLEU 3-gram: 0.000000
Corpus BLEU 4-gram: 0.000000
METEOR: 0.379528


test_best_multi_attn_best_MULTI.hypo
TEST
rouge-1
r: 0.5688160508107216
p: 0.33706890414542284
f: 0.41499642528639163
rouge-2
r: 0.2374550157172321
p: 0.12811426409120022
f: 0.162179031722

In [7]:
hypotheses, reference = load_ref_and_hypo(test_hypo_pretrained, tokenize=True)

In [14]:
reference = [[ref] for ref in reference]

In [15]:
from datasets import load_metric
metric = load_metric("bleu")

In [16]:
metric.add_batch(predictions=hypotheses, references=reference)

In [19]:
score = metric.compute(predictions=hypotheses, references=reference)

In [20]:
score

{'bleu': 0.07378321415307464,
 'precisions': [0.24236798679867988,
  0.09498718313870692,
  0.04909923213230951,
  0.02621895124195032],
 'brevity_penalty': 1.0,
 'length_ratio': 1.5237297014143532,
 'translation_length': 14544,
 'reference_length': 9545}

In [33]:
metric = load_metric("sacrebleu")
hypotheses, reference = load_ref_and_hypo(test_hypo_pretrained)
reference = [[ref] for ref in reference]

In [34]:
score = metric.compute(predictions=hypotheses, references=reference)
score

{'score': 12.548407269194492,
 'counts': [6011, 2789, 1528, 773],
 'totals': [17570, 17070, 16570, 16070],
 'precisions': [34.21172453044963,
  16.3386057410662,
  9.221484610742305,
  4.8102053515868075],
 'bp': 1.0,
 'sys_len': 17570,
 'ref_len': 12718}

In [39]:
score = metric.compute(predictions=[hypotheses[0]], references=[reference[0]])
score

{'score': 2.1835719620791547,
 'counts': [13, 2, 0, 0],
 'totals': [63, 62, 61, 60],
 'precisions': [20.634920634920636,
  3.225806451612903,
  0.819672131147541,
  0.4166666666666667],
 'bp': 1.0,
 'sys_len': 63,
 'ref_len': 31}